In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


# standard libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pandas import DataFrame

# different metrics and scoring methods
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_curve,
    auc,
)
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

# import models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# for hyperparameter tuning
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

from imblearn.over_sampling import SMOTE

# Import Data

For Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Pfad zur CSV-Datei
file_path = '/content/drive/My Drive/final_df.csv'

# Datei laden
flights = pd.read_csv(file_path)

flights

From local folder:

In [ ]:
file_path = "/Users/elocher/Business/Project/data/"

flights = pd.read_csv(file_path + "final_df.csv")

In [5]:
flights = pd.read_csv('../data/clean/final_df.csv')

# 0. Data Inspection

In [ ]:
ls = []
for i in flights.columns:
  ls.append(i)

In [ ]:
ls

In [ ]:
for el in ls:
  print(flights[el])

## Discussion with Raphael about Feature selection

- Departure Delay und Arrival Delay herausnehmen (da Summe Delayarten = WeatherDelay)
- Diverted und Cancelled (bei Regression herausnehmen oder bei Classification: bei Diverted oder Cancelles bei Arrivel_Time_Datetime die Scheduled_Arrival_time hineingetan -> entweder die Arrival Time herausnehmen


- Departure_Delay, Arrival_Delay, Elapsed_time, Air_time, Taxi_in, Taxi_out = 0, wenn Diverted oder Cancelled)
- Bei Departure_time und Arrival_time wurden Scheduled_Arrival_time und Scheduled_Departure_time hineingetan, falls Diverted oder Cancelled
-> Bei Random Forest müsste man zuerst predicten, ob er Cancelled ist oder nicht, wenn nicht Cancelled, dann Delayed oder nicht Delayed. Und falls Delayed, welche Klasse.

- Aus Call: Timelaps aus Wetter: bei allen Features dokumentieren, welche dieser Features relevant sind und welche nicht.
- In Notebook: lange Codezelle vor "Missing Data Dummies und Feature Engineering" bei Merged-Dataset. In Zellen 

- Sheduled_Time ist in Minuten -> zeigt Differenz zwischen Scheduled_Departure_time und Scheduled_Arrival_time -> unter Berücksichtigung der Zeitdifferenz
- Elapsed_time = dasselbe für die Arrival_time und Departure_time, also nicht scheduled.

- Taxi_in + Taxi_out + Flight-time = Elapsed_time

**In Dataset drinnlassen:**
- Airline-Boolean drinnlassen
- Airport-Boolean drinnlassen
- Alle Weather_data -> dann Feature_importance
- Flight-average weather delay -> drinnlassen -> groupby Flight_number und column WeatherDelay mean -> by flight_number gemapped auf Average_weather_delay
im Schnitt 100 Flüge aus denen Average entsteht.
- Weather beider airports -> Origin und DESTINATION_AIRPORT
- Weekend Boolean drinnlassen
- (Day of the Week-Dummy, evt.)

**Wetterdaten Metrik:**
- sped = Windspeed mph
- p01m = Presipitation in mm
- tempc = airtemp in celsius
- visibility in miles

In [6]:
columns_to_drop = ['DEPARTURE_DELAY',
                    'TAXI_OUT',
                    'SCHEDULED_TIME',
                    'ELAPSED_TIME',
                    'AIR_TIME',
                    'TAXI_IN',
                    'ARRIVAL_DELAY',
                    'SECURITY_DELAY',
                    'LATE_AIRCRAFT_DELAY',
                    'SCHEDULED_DEPARTURE_DATETIME',
                    'DEPARTURE_TIME_DATETIME',
                    'ARRIVAL_TIME_DATETIME',
                    'SCHEDULED_ARRIVAL_DATETIME',
                    'departure_6hr_before',
                    'departure_1hr_before',
                    'departure_1hr_after',
                    'departure_6hr_after',
                    'arrival_6hr_before',
                    'arrival_1hr_before',
                    'arrival_1hr_after',
                    'arrival_6hr_after',
                    'CANCELLATION_DESCRIPTION_National Air System',
                    'CANCELLATION_DESCRIPTION_Not cancelled',
                    'CANCELLATION_DESCRIPTION_Weather',
                    'DIVERTED',
                    'CANCELLED',
                    'AIR_SYSTEM_DELAY',
                    'AIRLINE_DELAY']

flights = flights.drop(columns=columns_to_drop)

In [7]:
pd.DataFrame(flights['WEATHER_DELAY'] > 0).value_counts()

WEATHER_DELAY
False            453636
True               6754
Name: count, dtype: int64

Bei einer binären Klassifizierung von WEATHER_DELAY haben wir es mit einem großen Ungleichgewicht zwischen den beiden Klassen zu tun (453k vs. 6.7k). Wir können eine einfache logistische Regression oder XGBClassification versuchen, um zu sehen, ob wir eine gute Leistung erzielen oder nicht. Wir wählen zweiteres Modell vor dem RandomForest, da wir vor allem aus Assignment 2 und 3 sowie aus "Fundamentals and Methods of CS" wissen, dass das Hyperparameter-Tuning bei einem XGBClassifier Modell besser funktioniert.

## Feature vs. Label und Feature vs. Feature Analyse

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Alle Spalten außer WEATHER_DELAY identifizieren
features = flights.columns.difference(['WEATHER_DELAY'])

# Subplots für mehrere Plots
num_features = len(features)
cols = 4  # Anzahl der Spalten in der Grid-Darstellung
rows = (num_features + cols - 1) // cols  # Automatische Berechnung der Zeilen

fig, axes = plt.subplots(rows, cols, figsize=(20, rows * 4))
axes = axes.flatten()  # Um leichter iterieren zu können

for i, feature in enumerate(features):
    sns.scatterplot(data=flights, x=feature, y='WEATHER_DELAY', ax=axes[i], alpha=0.5)
    axes[i].set_title(f"{feature} vs WEATHER_DELAY")
    axes[i].grid()

# Überschüssige Subplots deaktivieren
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()


**Einige Punkte zu obigen Korrelationsplots:**

1. Wir sehen, dass wir einige kategoriale Features haben. Manche sind bereits als Boolean formatiert (z.B. die verschiedenen Ablugs- und Ankunftsflughafen). Bei anderen stellt sich die Frage, ob es nicht mehr Sinn machen würde, sie als boolean zu formatieren. Dazu zählt das Feature `visibility (vsby)`, welches jeweils am Ankunfts- und Abflugsflughafen zur Abflugs-/Ankunftszeit, 1h vor- und danach sowie 6h vor- und danach gezeigt wird (dies wurde bei allen Wetterdaten so gemacht). Wir sehen hierbei, dass eine schlechtere `visibility` den `Weather_Delay` tendenziell steigen lässt (höherer Wert bei visibility führt zu höherer wetterbedingten Verspätung). Deswegen macht es Sinn, dieses Feature als numerischen Wert im Dataset zu lassen, damit diese Information nicht verloren geht.
2. Beim Feature `Percipitation (p01m)`, Niederschlag, ist zu erkennen, dass mehr Niederschlag nicht zu mehr Verspätung führt, sondern tendenziell bei einer höheren Niederschlagsmenge in mm keine Verspätungen zu sehen sind. Hingegen sehen wir bei extrem viele wetterbedingte Verspätungen, wenn es 0 mm oder nur ganz wenig Niederschlag gibt. Dies entspricht nicht unbedingt unseren Erwartungen, da wir dachten, dass mehr Niederschlag den Flugverkehr behindern könnte. Besonders wenn der Niederschlag mit tiefen Minustemperaturen verbunden ist (also Schneefall).
3. Bei der `Windgeschwindigkeit (sped)` in mph sehen wir, dass es doch einige Flüge gibt, die Verspätungen aufzeigen, wenn der Wind blässt. Dennoch ist auch hier zu sehen, dass dieses Feature alleine nicht sehr gut die wetterbedingten Verspätungen erklären kann (bsp. gibt es viele Flüge mit wetterbedingten Verspätungen trotz 0 mph Wind)
4. Beim Feature `Temperatur (tmpc)` in Grad Celsius gibt es keinen erkennbaren linearen Zusammenhang zwischen der Temperatur und den wetterbedingten Verspätungen. Die Datenpunkte sind stark gestreut, was auf eine geringe bis gar keine lineare Korrelation hinweist. Auch hier zeigt ein grossteil der Flüge keine wetterbedingte Verspätung (y=0). Dies deutet darauf hin, dass wetterbedingte Verspätungen nicht ausschließlich von der Temperatur abhängen.
5. `Durchschnittliche Verspätung pro Flugnummer (Average_Weather_Delay)`: Die meisten Flüge haben sehr niedrige Werte für die durchschnittliche wetterbedingte Verspätung pro Flugnummer (< 10) und die tatsächliche wetterbedingte Verspätung des einzelnen Flugs (y ≈ 0). Dies deutet darauf hin, dass wetterbedingte Verspätungen in der Regel selten auftreten und oft nur in geringer Höhe. Es gibt zudem eine dichte Clusterbildung im Bereich x < 10 und y < 200, was zeigt, dass Flüge mit niedrigen durchschnittlichen wetterbedingten Verspätungen meist ebenfalls niedrige Verspätungen für den einzelnen Flug aufweisen. Für Flüge mit durchschnittlichen wetterbedingten Verspätungen > 20  ist die Streuung der tatsächlichen wetterbedingten Verspätungen grösser (bis zu 1000 Minuten). Dies könnte darauf hinweisen, dass eine höhere durchschnittliche Verspätung pro Flugnummer auch das Risiko für grosse Verspätungen bei einzelnen Flügen erhöht.

**Fazit:**
Ein Grossteil der Flüge weist keine wetterbedingten Verspätungen auf. Selbst wenn wetterbezogene Faktoren wie Sicht, Wind oder Temperatur in ungünstigen Bereichen liegen, bleibt der Anteil der betroffenen Flüge gering. Viele der analysierten Wettermerkmale (z. B. Niederschlag, Wind, Temperatur) weisen nur schwache oder keine lineare Korrelation mit wetterbedingten Verspätungen auf. Dies deutet darauf hin, dass wetterbedingte Verspätungen durch eine Kombination mehrerer Faktoren entstehen. Das liefert Hinweise darauf, dass wir evt. mit nicht-linearen Modellen bessere Ergebnisse erzielen könnten.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Korrelationsmatrix berechnen
correlation_matrix = flights.corr()

# Heatmap der Korrelationsmatrix erstellen
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Korrelation zwischen Features (inkl. WEATHER_DELAY)")
plt.show()


**Multikollinearität:**

Wir sehen zuerst einmal, dass unsere Timelags bei den Wetterdaten z.T. sehr stark miteinander korrellieren. Dies kann zu Multikollinearität führen. Multikollinearität tritt auf, wenn zwei oder mehr Features in einem Datensatz hoch miteinander korreliert sind. Das bedeutet, dass sie eine sehr ähnliche Information liefern, was die Fähigkeit eines Modells, den Einfluss einzelner Features richtig zu schätzen, beeinträchtigen kann.

Bei Modellen wie der **logistischen Regression** oder **linearen Regression** kann Multikollinearität zu Problemen führen, weil sie die Koeffizienteninstabilität verursacht. Dies bedeutet, dass es schwierig wird, den individuellen Einfluss der korrelierten Features auf das Label korrekt zu bewerten. Die Schätzwerte der Regressionskoeffizienten können verzerrt oder hochgradig variabel sein.

**Random Forest** und andere **baum-basierte Modelle** sind weniger anfällig für Probleme der Multikollinearität, da diese Modelle nichtlinear und hierarchisch arbeiten. Die Bäume lernen, die wichtigsten Merkmale zu trennen und zu kombinieren, ohne dass sie durch starke Korrelationen zwischen Features stark beeinträchtigt werden.
Das bedeutet, dass man sich bei der Verwendung von Modellen wie Random Forest oder Gradient Boosting in der Regel weniger Sorge um Multikollinearität machen muss. Diese Modelle können weiterhin nützliche Muster erkennen, auch wenn einige Features stark miteinander korrelieren.

In [ ]:
# Nur Features mit signifikanter Korrelation zum Label (z. B. |Korrelation| > 0.01)
threshold = 0.01
correlation_matrix = flights.corr()

# Filter für hohe Korrelation mit WEATHER_DELAY
high_corr_features = correlation_matrix.loc[
    correlation_matrix['WEATHER_DELAY'].abs() > threshold, 
    correlation_matrix['WEATHER_DELAY'].abs() > threshold
]

# Heatmap mit reduzierter Anzahl von Features
plt.figure(figsize=(10, 8))
sns.heatmap(high_corr_features, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Heatmap der Features mit hoher Korrelation zum Label (|r| > 0.01)")
plt.show()


**Interpretation der höchsten Feature-Label Korrelation:**

Ein sehr geringer Korrelationskoeffizient (nahe null) zwischen den meisten Features und dem Ziel (Label). Dies deutet darauf hin, dass viele dieser Features wenig bis gar keinen Einfluss auf das Label haben. Diese irrelevanten Features könnten das Modell unnötig komplex machen und die Rechenleistung erhöhen, ohne eine bessere Leistung zu liefern.

Für unsere Modelle bedeutet dies:
+ Der **Random Forest** benötigt keine explizite Feature-Auswahl, da er während des Trainings selbst entscheidet, welche Features am nützlichsten sind. Features mit einer geringen Korrelation zu WEATHER_DELAY werden weniger wichtig sein und könnten in den Entscheidungsbäumen weniger häufig ausgewählt werden.
Trotzdem könnte es hilfreich sein, die Feature-Auswahl zu optimieren, um die Rechenkosten zu reduzieren und das Modell effizienter zu machen.
+ Bei der **logistischen/linearen Regression** sind Features mit sehr geringer Korrelation zu WEATHER_DELAY möglicherweise nicht signifikant und könnten zu einer schlechteren Modellleistung führen, wenn sie im Modell verbleiben. Wir werden hier versuchen, irrelevante Features zu entfernen oder eine L1-Regularisierung (Lasso) zu verwenden, die automatisch weniger wichtige Features eliminiert.

## Label Analyse

In [ ]:
flights['WEATHER_DELAY'].value_counts()

Bereits hier sehen wir, dass wir in über 450k der Flüge kein Weather Delay haben. Eine Häufigkeitsverteilung des gesamten Labels macht also nicht viel Sinn (vgl. Plot unten).

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 4))

# Draw the histogram
ax.hist(flights["WEATHER_DELAY"], bins=1000)

# Draw the mean and median lines
ymax = 5000  # Maximum of the y-axis
ax.vlines([flights["WEATHER_DELAY"].mean()], ymin=0, ymax=ymax, linestyles="dashed", 
          colors=["red"], label="Mean")
ax.vlines([flights["WEATHER_DELAY"].median()], ymin=0, ymax=ymax, linestyles="dashed",
            colors=["green"], label="Median")

# Draw 0.25, 0.75 quantiles
ax.vlines([flights["WEATHER_DELAY"].quantile(0.25), flights["WEATHER_DELAY"].quantile(0.75)], ymin=0, ymax=ymax,
            linestyles="dashed", colors=["orange", "orange"], label="Quantiles (0.25, 0.75)")

# Set the x-axis limit
ax.set_xlim(0, 20)  # Limit the x-axis from 0 to 100

# Add the legend
ax.legend()

# Set labels and title
ax.set_xlabel("WEATHER_DELAY")
ax.set_ylabel("Frequency")
ax.set_title("Weather Delay Histogram")

# Show the plot
plt.show()

# Display descriptive statistics of WEATHER_DELAY
flights["WEATHER_DELAY"].describe()


Deswegen plotten wir nun eine Verteilung aller wetterbedingten Flugverspätungen (dh. WEATHER_DELAY > 0).

In [ ]:
import matplotlib.pyplot as plt

# Filtere nur die Werte größer als 0 für die Anzeige
weather_delay_nonzero = flights[flights["WEATHER_DELAY"] > 0]["WEATHER_DELAY"]

fig, ax = plt.subplots(figsize=(8, 4))

# Zeichne das Histogramm nur für Werte > 0
ax.hist(weather_delay_nonzero, bins=100, color='blue', alpha=0.7)

# Berechne den maximalen Wert der Y-Achse
ymax = 5000  # Du kannst das ymax weiter anpassen

# Zeichne die Mittelwert- und Median-Linien
ax.vlines([weather_delay_nonzero.mean()], ymin=0, ymax=ymax, linestyles="dashed", 
          colors=["red"], label="Mean")
ax.vlines([weather_delay_nonzero.median()], ymin=0, ymax=ymax, linestyles="dashed",
            colors=["green"], label="Median")

# Zeichne die 0.25 und 0.75 Quantile
ax.vlines([weather_delay_nonzero.quantile(0.25), weather_delay_nonzero.quantile(0.75)], ymin=0, ymax=ymax,
            linestyles="dashed", colors=["orange", "orange"], label="Quantiles (0.25, 0.75)")

# Setze das x-Achsenlimit (nur für Werte größer als 0)
ax.set_xlim(0, 200)  # Limit der x-Achse

# Füge die Legende hinzu
ax.legend()

# Setze Beschriftungen und Titel
ax.set_xlabel("WEATHER_DELAY")
ax.set_ylabel("Frequency")
ax.set_title("Weather Delay Histogram (without 0)")

# Zeige den Plot
plt.show()

# Zeige deskriptive Statistiken der WEATHER_DELAY Spalte
weather_delay_nonzero.describe()


An diesem Punkt haben wir uns gefragt, ob es Sinn macht, mit einem solchen Dataset weiterzuarbeiten. Eine Regression für die wetterbedingten Verspätungen würde durch das oben gesehene Ungleichgewicht der Datenverteilung komplett verzerrt werden. Da wir jedoch an unserem Use-Case festhalten wollten, kamen wir auf folgende Idee:

1. Binäres Klassifikationsmodell für wetterbedingte Verspätung: Sagt vorher, ob eine Verspätung auftritt oder nicht (binäre Klassifikation: 0 oder >0).
2. Regressionsmodell für wetterbedingte Verspätungen: Falls eine Wetterbedingte Verspätung vorhergesagt wurde, wird eine Regression durchgeführt, die vorhersagt, wie gross diese Verspätung sein wird.


# 1. Binäre Klassifikation

In [28]:
X = flights.drop(columns=["WEATHER_DELAY"])
y = flights["WEATHER_DELAY"]

In [29]:
X

,DISTANCE,SCHEDULED_DEPARTURE_DATETIME_tmpc,SCHEDULED_DEPARTURE_DATETIME_sped,SCHEDULED_DEPARTURE_DATETIME_p01m,SCHEDULED_DEPARTURE_DATETIME_vsby,departure_6hr_before_tmpc,departure_6hr_before_sped,departure_6hr_before_p01m,departure_6hr_before_vsby,departure_1hr_before_tmpc,...,Destination_AIRPORT_Dallas/Fort Worth International Airport,Destination_AIRPORT_Denver International Airport,Destination_AIRPORT_George Bush Intercontinental Airport,Destination_AIRPORT_Hartsfield-Jackson Atlanta International Airport,Destination_AIRPORT_LaGuardia Airport (Marine Air Terminal),Destination_AIRPORT_Los Angeles International Airport,Destination_AIRPORT_McCarran International Airport,Destination_AIRPORT_Phoenix Sky Harbor International Airport,Destination_AIRPORT_San Francisco International Airport,WEEKEND
0,731,13.33,4.60,0.00,10.0,0.00,0.00,0.0000,10.0,11.67,...,True,False,False,False,False,False,False,False,False,0
1,762,13.89,3.45,0.00,10.0,2.78,0.00,0.0000,10.0,13.33,...,False,False,False,False,True,False,False,False,False,0
2,689,13.33,3.45,0.00,10.0,6.11,0.00,0.0000,10.0,13.89,...,False,False,True,False,False,False,False,False,False,0
3,2139,7.22,4.60,0.25,10.0,13.33,3.45,0.0000,10.0,8.33,...,False,False,False,False,False,False,False,False,True,0
4,762,7.22,3.45,1.27,8.0,6.67,0.00,0.0001,10.0,7.22,...,False,False,False,False,True,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460385,2139,8.33,6.90,0.00,10.0,8.33,6.90,0.0000,10.0,8.33,...,False,False,False,True,False,False,False,False,False,0
460386,414,8.33,6.90,0.00,10.0,8.33,6.90,0.0000,10.0,8.33,...,False,False,False,False,False,False,True,False,False,0
460387,1846,8.33,6.90,0.00,10.0,8.33,6.90,0.0000,10.0,8.33,...,False,False,False,False,False,False,False,False,False,0
460388,2139,8.33,6.90,0.00,10.0,8.33,6.90,0.0000,10.0,8.33,...,False,False,False,True,False,False,False,False,False,0


In [30]:
y = y.apply(lambda x: 0.0 if x == 0 else 1.0)

In [31]:
y.value_counts()

WEATHER_DELAY
0.0    453636
1.0      6754
Name: count, dtype: int64

In [32]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460390 entries, 0 to 460389
Data columns (total 71 columns):
 #   Column                                                                Non-Null Count   Dtype  
---  ------                                                                --------------   -----  
 0   DISTANCE                                                              460390 non-null  int64  
 1   SCHEDULED_DEPARTURE_DATETIME_tmpc                                     460390 non-null  float64
 2   SCHEDULED_DEPARTURE_DATETIME_sped                                     460390 non-null  float64
 3   SCHEDULED_DEPARTURE_DATETIME_p01m                                     460390 non-null  float64
 4   SCHEDULED_DEPARTURE_DATETIME_vsby                                     460390 non-null  float64
 5   departure_6hr_before_tmpc                                             460390 non-null  float64
 6   departure_6hr_before_sped                                             460390 non-nul

In [33]:
# Split dataset into training and test sets
X['AVERAGE_WEATHER_DELAY'] = 0
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [34]:
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 322273 samples
Test set: 138117 samples


In [35]:
y_train.value_counts()

WEATHER_DELAY
0.0    317545
1.0      4728
Name: count, dtype: int64

In [36]:
y_test.value_counts()

WEATHER_DELAY
0.0    136091
1.0      2026
Name: count, dtype: int64

Durch `stratify` erreichen wir, dass die Klassenrepräsentation im Training und Test-set ungefähr denselben Anteil hat.

In [37]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 322273 entries, 254406 to 405927
Data columns (total 72 columns):
 #   Column                                                                Non-Null Count   Dtype  
---  ------                                                                --------------   -----  
 0   DISTANCE                                                              322273 non-null  int64  
 1   SCHEDULED_DEPARTURE_DATETIME_tmpc                                     322273 non-null  float64
 2   SCHEDULED_DEPARTURE_DATETIME_sped                                     322273 non-null  float64
 3   SCHEDULED_DEPARTURE_DATETIME_p01m                                     322273 non-null  float64
 4   SCHEDULED_DEPARTURE_DATETIME_vsby                                     322273 non-null  float64
 5   departure_6hr_before_tmpc                                             322273 non-null  float64
 6   departure_6hr_before_sped                                             322273 non-nul

In [38]:
X_train.columns

Index(['DISTANCE', 'SCHEDULED_DEPARTURE_DATETIME_tmpc',
       'SCHEDULED_DEPARTURE_DATETIME_sped',
       'SCHEDULED_DEPARTURE_DATETIME_p01m',
       'SCHEDULED_DEPARTURE_DATETIME_vsby', 'departure_6hr_before_tmpc',
       'departure_6hr_before_sped', 'departure_6hr_before_p01m',
       'departure_6hr_before_vsby', 'departure_1hr_before_tmpc',
       'departure_1hr_before_sped', 'departure_1hr_before_p01m',
       'departure_1hr_before_vsby', 'departure_1hr_after_tmpc',
       'departure_1hr_after_sped', 'departure_1hr_after_p01m',
       'departure_1hr_after_vsby', 'departure_6hr_after_tmpc',
       'departure_6hr_after_sped', 'departure_6hr_after_p01m',
       'departure_6hr_after_vsby', 'SCHEDULED_ARRIVAL_DATETIME_tmpc',
       'SCHEDULED_ARRIVAL_DATETIME_sped', 'SCHEDULED_ARRIVAL_DATETIME_p01m',
       'SCHEDULED_ARRIVAL_DATETIME_vsby', 'arrival_6hr_before_tmpc',
       'arrival_6hr_before_sped', 'arrival_6hr_before_p01m',
       'arrival_6hr_before_vsby', 'arrival_1hr_before_tmpc

In [39]:
# Apply OneHotEncoder to categorical columns, and scale numerical columns
numeric_features = [
                    'DISTANCE', 'SCHEDULED_DEPARTURE_DATETIME_tmpc',
                    'SCHEDULED_DEPARTURE_DATETIME_sped',
                    'SCHEDULED_DEPARTURE_DATETIME_p01m',
                    'SCHEDULED_DEPARTURE_DATETIME_vsby', 'departure_6hr_before_tmpc',
                    'departure_6hr_before_sped', 'departure_6hr_before_p01m',
                    'departure_6hr_before_vsby', 'departure_1hr_before_tmpc',
                    'departure_1hr_before_sped', 'departure_1hr_before_p01m',
                    'departure_1hr_before_vsby', 'departure_1hr_after_tmpc',
                    'departure_1hr_after_sped', 'departure_1hr_after_p01m',
                    'departure_1hr_after_vsby', 'departure_6hr_after_tmpc',
                    'departure_6hr_after_sped', 'departure_6hr_after_p01m',
                    'departure_6hr_after_vsby', 'SCHEDULED_ARRIVAL_DATETIME_tmpc',
                    'SCHEDULED_ARRIVAL_DATETIME_sped', 'SCHEDULED_ARRIVAL_DATETIME_p01m',
                    'SCHEDULED_ARRIVAL_DATETIME_vsby', 'arrival_6hr_before_tmpc',
                    'arrival_6hr_before_sped', 'arrival_6hr_before_p01m',
                    'arrival_6hr_before_vsby', 'arrival_1hr_before_tmpc',
                    'arrival_1hr_before_sped', 'arrival_1hr_before_p01m',
                    'arrival_1hr_before_vsby', 'arrival_1hr_after_tmpc',
                    'arrival_1hr_after_sped', 'arrival_1hr_after_p01m',
                    'arrival_1hr_after_vsby', 'arrival_6hr_after_tmpc',
                    'arrival_6hr_after_sped', 'arrival_6hr_after_p01m',
                    'arrival_6hr_after_vsby', 'AVERAGE_WEATHER_DELAY'
]
categorical_features = [
    'WEEKEND'
]

# take remaining features (because they have already been processed)
remaining_features_test = [col for col in X_test.columns if col not in numeric_features + categorical_features]
remaining_features_train = [col for col in X_train.columns if col not in numeric_features + categorical_features]
# create a DataFrame with the already processed features
X_remaining_test = X_test[remaining_features_test].reset_index(drop=True)
X_remaining_train = X_train[remaining_features_train].reset_index(drop=True)


In [40]:


# Process numeric features: scaling
scaler = StandardScaler()
X_train_numeric = X_train[numeric_features]
X_test_numeric = X_test[numeric_features]


In [42]:
X_train_numeric.columns

Index(['DISTANCE', 'SCHEDULED_DEPARTURE_DATETIME_tmpc',
       'SCHEDULED_DEPARTURE_DATETIME_sped',
       'SCHEDULED_DEPARTURE_DATETIME_p01m',
       'SCHEDULED_DEPARTURE_DATETIME_vsby', 'departure_6hr_before_tmpc',
       'departure_6hr_before_sped', 'departure_6hr_before_p01m',
       'departure_6hr_before_vsby', 'departure_1hr_before_tmpc',
       'departure_1hr_before_sped', 'departure_1hr_before_p01m',
       'departure_1hr_before_vsby', 'departure_1hr_after_tmpc',
       'departure_1hr_after_sped', 'departure_1hr_after_p01m',
       'departure_1hr_after_vsby', 'departure_6hr_after_tmpc',
       'departure_6hr_after_sped', 'departure_6hr_after_p01m',
       'departure_6hr_after_vsby', 'SCHEDULED_ARRIVAL_DATETIME_tmpc',
       'SCHEDULED_ARRIVAL_DATETIME_sped', 'SCHEDULED_ARRIVAL_DATETIME_p01m',
       'SCHEDULED_ARRIVAL_DATETIME_vsby', 'arrival_6hr_before_tmpc',
       'arrival_6hr_before_sped', 'arrival_6hr_before_p01m',
       'arrival_6hr_before_vsby', 'arrival_1hr_before_tmpc

In [43]:

X_train_scaled = scaler.fit_transform(X_train_numeric)
X_test_scaled = scaler.transform(X_test_numeric)

# Process categorical features: dummy coding
encoder = OneHotEncoder(handle_unknown="ignore", drop="first")
X_train_categorical = X_train[categorical_features]
X_test_categorical = X_test[categorical_features]
X_train_encoded = encoder.fit_transform(X_train_categorical).toarray()
X_test_encoded = encoder.transform(X_test_categorical).toarray()



In [44]:
# Combine numeric and categorical features back
X_train_encoded = pd.DataFrame(X_train_encoded, columns= encoder.get_feature_names_out())
X_test_encoded = pd.DataFrame(X_test_encoded, columns= encoder.get_feature_names_out())
X_train_scaled = pd.DataFrame(X_train_scaled, columns= scaler.get_feature_names_out())
X_test_scaled = pd.DataFrame(X_test_scaled, columns= scaler.get_feature_names_out())

X_train_processed = pd.concat([X_train_scaled, X_train_encoded, X_remaining_train], axis=1)
X_test_processed = pd.concat([X_test_scaled, X_test_encoded, X_remaining_test], axis=1)

# Check the result
print(f"X_train_processed shape: {X_train_processed.shape}")
print(f"X_test_processed shape: {X_test_processed.shape}")

X_train_processed shape: (322273, 72)
X_test_processed shape: (138117, 72)


In [45]:
X_train_processed.columns

Index(['DISTANCE', 'SCHEDULED_DEPARTURE_DATETIME_tmpc',
       'SCHEDULED_DEPARTURE_DATETIME_sped',
       'SCHEDULED_DEPARTURE_DATETIME_p01m',
       'SCHEDULED_DEPARTURE_DATETIME_vsby', 'departure_6hr_before_tmpc',
       'departure_6hr_before_sped', 'departure_6hr_before_p01m',
       'departure_6hr_before_vsby', 'departure_1hr_before_tmpc',
       'departure_1hr_before_sped', 'departure_1hr_before_p01m',
       'departure_1hr_before_vsby', 'departure_1hr_after_tmpc',
       'departure_1hr_after_sped', 'departure_1hr_after_p01m',
       'departure_1hr_after_vsby', 'departure_6hr_after_tmpc',
       'departure_6hr_after_sped', 'departure_6hr_after_p01m',
       'departure_6hr_after_vsby', 'SCHEDULED_ARRIVAL_DATETIME_tmpc',
       'SCHEDULED_ARRIVAL_DATETIME_sped', 'SCHEDULED_ARRIVAL_DATETIME_p01m',
       'SCHEDULED_ARRIVAL_DATETIME_vsby', 'arrival_6hr_before_tmpc',
       'arrival_6hr_before_sped', 'arrival_6hr_before_p01m',
       'arrival_6hr_before_vsby', 'arrival_1hr_before_tmpc

Training set: 322273 samples

Test set: 138117 samples

# Models

## Functions

In [ ]:
# We are going to create many models, let us keep track of their performance
# We will use a dataframe to store the results of the different models
results2 = pd.DataFrame({"Model": [], "Train_CV": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": [], "Best Threshold (J-statistic)": []})

In [ ]:
# We create a scorer that computes the given scores (accuracy, precision, recall and f1-score) 
def make_scorefunc(loss_func, greater_is_better=True):
    def scorer(y, y_pred):
        return loss_func(y, y_pred)
    return make_scorer(scorer, greater_is_better=greater_is_better)

In [ ]:
# Create the scorer dictionaries
scorer = {
    "Accuracy": make_scorefunc(accuracy),
    "Precision": make_scorefunc(precision),
    "Recall": make_scorefunc(recall),
    "F1-Score": make_scorefunc(f1)
}

In [ ]:
## Function to calculate the metrics and display the confusion matrix (based on the j-statistic threshold) and roc curve

def compute_metrics(model, X_train, y_train, X_test, y_test, scorer, model_name=None):
    # Extract the names from the scorer dictionary
    scorer_names = list(scorer.keys())

    # Fit the model to the training set
    model.fit(X_train, y_train)
    
    ### Visualization and Metric Calculation ###
    # Predict probabilities (for ROC curve and custom thresholding)
    y_probs = model.predict_proba(X_test)[:, 1]

    # Compute ROC curve and thresholds
    fpr, tpr, thresholds = roc_curve(y_test, y_probs)

    # Calculate the J-statistic and find the optimal threshold
    j_statistic = tpr - fpr
    optimal_idx = j_statistic.argmax()
    optimal_threshold = thresholds[optimal_idx]

    # Predict labels using the optimal threshold
    y_pred_optimal = (y_probs >= optimal_threshold).astype(int)

    # Create a dictionary for the pandas dataframe
    test_scores = {"Model": model_name, "Train_CV": "Test", "Best Threshold (J-statistic)": optimal_threshold}
    
    # Iterate over the scorers, now using y_test and y_pred_optimal
    for k in scorer_names:
        score_func = scorer[k]._score_func  # Extract the scoring function
        test_scores[k] = score_func(y_test, y_pred_optimal)  # Use y_pred_optimal

    # Append the results to the dataframe
    test_scores = pd.DataFrame(test_scores, index=[0])

    # Compute confusion matrix for the optimal threshold
    cm = confusion_matrix(y_test, y_pred_optimal)

    # Compute ROC area
    roc_auc = auc(fpr, tpr)

    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=ax1, cmap='Blues', colorbar=False)
    ax1.set_title('Confusion Matrix (Optimal Threshold)')
    ax1.set_xlabel('Predicted Label')
    ax1.set_ylabel('True Label')
    ax1.grid(False)

    # Calculate percentages for the confusion matrix
    cm_percent = cm / cm.sum() * 100  # Divide each cell by the total sum of the matrix to get percentages

    # Annotate with percentages below the numbers
    for i in range(cm.shape[0]):  # Iterate over rows
        for j in range(cm.shape[1]):  # Iterate over columns
            percentage = f"{cm_percent[i, j]:.1f}%"  # Format percentage
            ax1.text(j, i + 0.3, percentage, ha="center", va="center", fontsize=10, color="black")  # Add below numbers

    # Plot ROC curve
    ax2.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax2.scatter(fpr[optimal_idx], tpr[optimal_idx], color="red", 
                label=f"Optimal Threshold: {optimal_threshold:.5f}", zorder=5)
    ax2.set_xlim([0.0, 1.0])
    ax2.set_ylim([0.0, 1.05])
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('Receiver Operating Characteristic (ROC) Curve')
    ax2.legend(loc='lower right')

    # Adjust layout and display the plots
    plt.tight_layout()
    plt.show()

    return test_scores

In [ ]:
# function to calculate the metrics and display the confusion matrix (based on a self-defined threshold) and roc curve

def compute_metrics2(model, X_train, y_train, X_test, y_test, scorer, model_name=None, optimal_threshold=0.5):
    # Extract the names from the scorer dictionary
    scorer_names = list(scorer.keys())

    # Fit the model to the training set
    model.fit(X_train, y_train)
    
    ### Visualization and Metric Calculation ###
    # Predict probabilities (for ROC curve and custom thresholding)
    y_probs = model.predict_proba(X_test)[:, 1]

    # Compute ROC curve and thresholds
    fpr, tpr, thresholds = roc_curve(y_test, y_probs)


    # Find the index of the threshold closest to the optimal threshold
    optimal_idx = np.argmin(np.abs(thresholds - optimal_threshold))

    # Predict labels using the optimal threshold
    y_pred_optimal = (y_probs >= optimal_threshold).astype(int)

    # Create a dictionary for the pandas dataframe
    test_scores = {"Model": model_name, "Train_CV": "Test", "Best Threshold (J-statistic)": optimal_threshold}
    
    # Calculate the metrics for the optimal threshold
    test_scores["Accuracy"] = accuracy(y_test, y_pred_optimal)
    test_scores["Precision"] = precision(y_test, y_pred_optimal)
    test_scores["Recall"] = recall(y_test, y_pred_optimal)
    test_scores["F1-Score"] = f1(y_test, y_pred_optimal)

    # Append the results to the dataframe
    test_scores = pd.DataFrame(test_scores, index=[0])
    # Compute confusion matrix for the optimal threshold
    cm = confusion_matrix(y_test, y_pred_optimal)

    # Compute ROC area
    roc_auc = auc(fpr, tpr)

    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=ax1, cmap='Blues', colorbar=False)
    ax1.set_title('Confusion Matrix (Optimal Threshold)')
    ax1.set_xlabel('Predicted Label')
    ax1.set_ylabel('True Label')
    ax1.grid(False)

    # Calculate percentages for the confusion matrix
    cm_percent = cm / cm.sum() * 100  # Divide each cell by the total sum of the matrix to get percentages

    # Annotate with percentages below the numbers
    for i in range(cm.shape[0]):  # Iterate over rows
        for j in range(cm.shape[1]):  # Iterate over columns
            percentage = f"{cm_percent[i, j]:.1f}%"  # Format percentage
            ax1.text(j, i + 0.3, percentage, ha="center", va="center", fontsize=10, color="black")  # Add below numbers

    # Plot ROC curve
    ax2.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax2.scatter(fpr[optimal_idx], tpr[optimal_idx], color="red", 
                label=f"Optimal Threshold: {optimal_threshold:.5f}", zorder=5)
    ax2.set_xlim([0.0, 1.0])
    ax2.set_ylim([0.0, 1.05])
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('Receiver Operating Characteristic (ROC) Curve')
    ax2.legend(loc='lower right')

    # Adjust layout and display the plots
    plt.tight_layout()
    plt.show()

    return test_scores

## Logistic Regression

In [ ]:
log_reg1 = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)

In [ ]:
log_reg1_results = compute_metrics(log_reg1, X_train_processed, y_train, X_test_processed, y_test, scorer=scorer, model_name="Logistic Regression1")

In [ ]:
results2 = pd.concat([results2, log_reg1_results], axis=0)

**Anpassung des Thresholds:**

Da wir danach mit der vorhergesagten Klasse 1 (also Wetterverspätung), scheint die Zahl von False Positives (=fälschlicherweise vorhergesagt Wetterverspätung in 48'631 Fällen) zu hoch zu sein. Mit dem Anpassen des Thresholds nach oben würden wir diese Zahl herunterkriegen, weil dann es dann eine höhere Wahrscheinlichkeit bei den vorhersagen braucht, damit die Klasse 1 zugewiesen wird.

In [ ]:
log_reg1_results2 = compute_metrics2(log_reg1, X_train_processed, y_train, X_test_processed, y_test, scorer=scorer, model_name="Logistic Regression1 (new threshold)", optimal_threshold=0.55)

In [ ]:
results2 = pd.concat([results2, log_reg1_results2], axis=0)

In [ ]:
results2

### Approach mit Smote

In [ ]:
# SMOTE auf die Trainingsdaten anwenden
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_processed, y_train)

In [ ]:
log_reg3 = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)

In [ ]:
log_reg3_results = compute_metrics(log_reg3, X_train_balanced, y_train_balanced, X_test_processed, y_test, scorer=scorer, model_name="Logistic Regression 2 (SMOTE)")

In [ ]:
results2 = pd.concat([results2, log_reg3_results], axis=0)

## Random Forest

### BayesSearchCV

In [ ]:
# DO NOT RUN THIS CODE CELL AS IT TAKES MORE THAN 10 HOURS TO RUN!

# defining the parameter space for the Bayesian Optimization based on the above comments
param_space = {
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced'],
    'max_depth': (11, 18),  # around 15
    'max_features': ['sqrt', 'log2', 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2],  # Nur die gefundenen Werte
    'min_samples_leaf': (2, 8),  # 2 to 8
    'min_samples_split': (2, 9),  # 2 to 9
    'n_estimators': (450, 950),  # values between 450 and 950
}

# StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=74)

# build the model
forest_bs = RandomForestClassifier(random_state=42)

# BayesSearchCV
bayes_search = BayesSearchCV(
    estimator=forest_bs,
    search_spaces=param_space,
    n_iter=70,  # number of iterations
    scoring='roc_auc',  # ROC-AUC as metric
    cv=cv, # StratifiedKFold as before
    n_jobs=1, # set to 1 because parallel processing with bayesian optimization can cause issues
    verbose=2, # show progress
    random_state=42 # for reproducibility
)

# Training
bayes_search.fit(X_train_processed, y_train)

# results
print("Beste Hyperparameter aus BayesSearchCV:", bayes_search.best_params_)

## XGBoost (Gradient-Boosting)

### BayesSearchCV

In [ ]:
# parameter space for BayesSearchCV
param_space = {
    'booster': ['gbtree'],  # only 'gbtree' is used
    'colsample_bytree': Real(0.87, 0.995, prior='uniform'),  # range: 0.87 - 0.995
    'gamma': Real(0.014, 0.15, prior='uniform'),  # range: 0.014 - 0.15
    'learning_rate': Real(0.01, 0.06, prior='uniform'),  # range: 0.01 - 0.06
    'max_depth': Integer(3, 6),  # range: 3 - 6
    'min_child_weight': Integer(3, 8),  # range: 3 - 8
    'n_estimators': Integer(490, 760),  # range: 490 - 760
    'reg_alpha': Real(0.0002, 0.18, prior='uniform'),  # reange: 0.0002 - 0.18
    'reg_lambda': Real(0.05, 0.3, prior='uniform'),  # range: 0.05 - 0.3
    'scale_pos_weight': Integer(1, 5),  # range: 1 - 5
    'subsample': Real(0.78, 0.985, prior='uniform')  # range: 0.78 - 0.985
}

# define Basismodel
xgb_model = xgb.XGBClassifier(random_state=42)

# StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# create BayesSearchCV
bayes_search_xgb = BayesSearchCV(estimator=xgb_model,
                                search_spaces=param_space,
                                n_iter=70,  # number of random configuration that will be tested
                                cv=cv,  # StratifiedKFold as before
                                scoring='roc_auc',  # ROC-AUC as metric
                                n_jobs=1,  # set to 1 because parallel processing with bayesian optimization can cause issues
                                random_state=42,  # reproducibility
                                verbose=2,  # show progress
                                )

# fit BayesSearchCV on Train data
bayes_search_xgb.fit(X_train_processed, y_train)

# print best parameters
print("Beste Parameter aus BayesSearchCV:", bayes_search_xgb.best_params_)

Wir haben folgende Hyperparemter gefunden:

Beste Parameter aus BayesSearchCV: OrderedDict({'booster': 'gbtree', 'colsample_bytree': 0.995, 'gamma': 0.014, 'learning_rate': 0.04944699914723699, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 760, 'reg_alpha': 0.18, 'reg_lambda': 0.3, 'scale_pos_weight': 1, 'subsample': 0.78})

In [ ]:
# best Hyperparameters from BayesSearchCV
best_params1 = {
    'booster': 'gbtree',
    'colsample_bytree': 0.995,
    'gamma': 0.014,
    'learning_rate': 0.04944699914723699,
    'max_depth': 6,
    'min_child_weight': 3,
    'n_estimators': 760,
    'reg_alpha': 0.18,
    'reg_lambda': 0.3,
    'scale_pos_weight': 1,
    'subsample': 0.78
}

# Define XGBClassifier-Model with the best Hyperparameters
xgb_model_1 = xgb.XGBClassifier(
    random_state=42,  # Same random state for reproducibility
    **best_params1  # Add the best hyperparameters
)

In [ ]:
xgb_1_scores = compute_metrics(xgb_model_1, X_train_processed, y_train, X_test_processed, y_test, scorer=scorer, model_name="XGBClassifier 1 (BayesSearchCV)")

In [ ]:
results2 = pd.concat([results2, xgb_1_scores], axis=0)

In [ ]:
results2

In [ ]:
# parameter space for BayesSearchCV
param_space = {
    'booster': ['gbtree'],  # only 'gbtree' is used
    'colsample_bytree': Real(0.87, 0.995, prior='uniform'),  # range: 0.87 - 0.995
    'gamma': Real(0.014, 0.15, prior='uniform'),  # range: 0.014 - 0.15
    'learning_rate': Real(0.01, 0.06, prior='uniform'),  # range: 0.01 - 0.06
    'max_depth': Integer(3, 6),  # range: 3 - 6
    'min_child_weight': Integer(3, 8),  # range: 3 - 8
    'n_estimators': Integer(490, 760),  # range: 490 - 760
    'reg_alpha': Real(0.0002, 0.18, prior='uniform'),  # reange: 0.0002 - 0.18
    'reg_lambda': Real(0.05, 0.3, prior='uniform'),  # range: 0.05 - 0.3
    'scale_pos_weight': Integer(1, 5),  # range: 1 - 5
    'subsample': Real(0.78, 0.985, prior='uniform')  # range: 0.78 - 0.985
}

# define Basismodel
xgb_model = xgb.XGBClassifier(random_state=42)

# StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# create BayesSearchCV
bayes_search_xgb = BayesSearchCV(estimator=xgb_model,
                                search_spaces=param_space,
                                n_iter=70,  # number of random configuration that will be tested
                                cv=cv,  # StratifiedKFold as before
                                scoring='roc_auc',  # ROC-AUC as metric
                                n_jobs=1,  # set to 1 because parallel processing with bayesian optimization can cause issues
                                random_state=42,  # reproducibility
                                verbose=2,  # show progress
                                )

# fit BayesSearchCV on Train data
bayes_search_xgb.fit(X_train_balanced, y_train_balanced)

# print best parameters
print("Beste Parameter aus BayesSearchCV:", bayes_search_xgb.best_params_)

Beste Parameter aus BayesSearchCV: OrderedDict({'booster': 'gbtree', 'colsample_bytree': 0.995, 'gamma': 0.15, 'learning_rate': 0.06, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 760, 'reg_alpha': 0.02276324530745919, 'reg_lambda': 0.28188981298566596, 'scale_pos_weight': 1, 'subsample': 0.78})

In [ ]:
# best Hyperparameters from BayesSearchCV
best_params2 = {
    'booster': 'gbtree',
    'colsample_bytree': 0.995,
    'gamma': 0.15,
    'learning_rate': 0.06,
    'max_depth': 6,
    'min_child_weight': 3,
    'n_estimators': 760,
    'reg_alpha': 0.02276324530745919,
    'reg_lambda': 0.28188981298566596,
    'scale_pos_weight': 1,
    'subsample': 0.78
}

# Define XGBClassifier-Model with the best Hyperparameters
xgb_model_2 = xgb.XGBClassifier(
    random_state=42,  # Same random state for reproducibility
    **best_params2  # Add the best hyperparameters
)

In [ ]:
xgb_2_scores = compute_metrics(xgb_model_2, X_train_balanced, y_train_balanced, X_test_processed, y_test, scorer=scorer, model_name="XGBClassifier 2 (SMOTE)")

In [ ]:
results2 = pd.concat([results2, xgb_2_scores], axis=0)

In [ ]:
results2

# Neues Herangehensweise